In [1]:
import pandas as pd
from bicm import BipartiteGraph
import numpy as np
from tqdm import tqdm
import csv
import itertools 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import math

### Functions

In [2]:
def dump_degree_sequences(train,test,fold=0,unseen_folder='VecNet_Unseen_Nodes'):
    
    ligands = list(set(train['InChiKey'].tolist()))
    targets = list(set(train['target_aa_code'].tolist()))
    
    ligands_degree_dict = dict()

    for inchikey_chem in tqdm(ligands):
        sum_df = train[train['InChiKey'] == inchikey_chem]
        ligands_degree_dict[inchikey_chem] = dict()
        ligands_degree_dict[inchikey_chem]['deg_0'] = len(sum_df[sum_df['Y'] == 0])
        ligands_degree_dict[inchikey_chem]['deg_1'] = len(sum_df[sum_df['Y'] == 1])
        
    targets_degree_dict = dict()

    for aa_target in tqdm(targets):
        sum_df = train[train['target_aa_code'] == aa_target]
        targets_degree_dict[aa_target] = dict()
        targets_degree_dict[aa_target]['deg_0'] = len(sum_df[sum_df['Y'] == 0])
        targets_degree_dict[aa_target]['deg_1'] = len(sum_df[sum_df['Y'] == 1])
        
    degree_train_1_0_ligands = [ligands_degree_dict[key_val]['deg_1'] for key_val in tqdm(ligands_degree_dict.keys())]
    degree_train_0_1_ligands = [ligands_degree_dict[key_val]['deg_0'] for key_val in tqdm(ligands_degree_dict.keys())]

    degree_train_1_0_targets = [targets_degree_dict[key_val]['deg_1'] for key_val in tqdm(targets_degree_dict.keys())]
    degree_train_0_1_targets = [targets_degree_dict[key_val]['deg_0'] for key_val in tqdm(targets_degree_dict.keys())]
    
    with open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/degreetrain10ligands_' + str(fold) + '.txt', 'w') as file:
        for degree in degree_train_1_0_ligands:
            file.write("%i\n" % degree)
        
    file.close()
    
    with open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/degreetrain01ligands_' + str(fold) + '.txt', 'w') as file:
        for degree in degree_train_0_1_ligands:
            file.write("%i\n" % degree)
        
    file.close()
    
    with open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/degreetrain10targets_' + str(fold) + '.txt', 'w') as file:
        for degree in degree_train_1_0_targets:
            file.write("%i\n" % degree)
        
    file.close()
    
    with open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/degreetrain01targets_' + str(fold) + '.txt', 'w') as file:
        for degree in degree_train_0_1_targets:
            file.write("%i\n" % degree)
        
    file.close()
    
    textfile = open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/ligands_' + str(fold) + '.txt', "w")
    for element in ligands:
        textfile.write(element + "\n")
    textfile.close()
    
    textfile = open('../data/sars-busters-consolidated/GitData/' + unseen_folder + '/Degree_Sequences/targets_' + str(fold) + '.txt', "w")
    for element in targets:
        textfile.write(element + "\n")
    textfile.close()
    
    return 

In [3]:
def get_configuration_model_performance(train,test,ligand_file_path,target_file_path,summat10_file_path,summat01_file_path):
    
    text_file = open(ligand_file_path, "r") # Rows of the adjacency matrix in order
    ligands = text_file.readlines()

    text_file = open(target_file_path, "r") # Columns of the adjacency matrix in order 
    targets = text_file.readlines()
    
    ligands = [j.replace('\n','') for j in tqdm(ligands)]
    targets = [j.replace('\n','') for j in tqdm(targets)]
    
    number_ligands = len(ligands)
    number_targets = len(targets)
    
    train_pos = train[train['Y'] == 1]
    train_neg = train[train['Y'] == 0]
    
    pos_deg_0_ligands = []
    pos_deg_0_targets = []
    neg_deg_0_ligands = []
    neg_deg_0_targets = []
    
    ligand_degree_ratio = dict()
    ligand_all_average = []

    for ligand in tqdm(ligands):
        pos_deg = len(train_pos[train_pos['InChiKey'] == ligand])
        neg_deg = len(train_neg[train_neg['InChiKey'] == ligand])
        ligand_degree_ratio[ligand] = dict()
        ligand_degree_ratio[ligand]['deg_ratio'] = pos_deg / (pos_deg + neg_deg)
        ligand_degree_ratio[ligand]['deg_avg'] = pos_deg / number_targets 
        ligand_all_average.append(pos_deg / number_targets)
        if pos_deg == 0:
            pos_deg_0_ligands.append(ligand)
        if neg_deg == 0:
            neg_deg_0_ligands.append(ligand)
    
    ligands_all_avg = sum(ligand_all_average) / number_ligands
    
    targets_degree_ratio = dict()
    target_all_average = []

    for target in tqdm(targets):
        pos_deg = len(train_pos[train_pos['target_aa_code'] == target])
        neg_deg = len(train_neg[train_neg['target_aa_code'] == target])
        targets_degree_ratio[target] = dict()
        targets_degree_ratio[target]['deg_ratio'] = pos_deg / (pos_deg + neg_deg)
        targets_degree_ratio[target]['deg_avg'] = pos_deg / number_ligands
        target_all_average.append(pos_deg / number_ligands)
        if pos_deg == 0:
            pos_deg_0_targets.append(target)
        if neg_deg == 0:
            neg_deg_0_targets.append(target)
    
    targets_all_avg = sum(target_all_average) / number_targets
    
    print('Ligands with positive degree 0: ',len(pos_deg_0_ligands))
    print('Ligands with negative degree 0: ',len(neg_deg_0_ligands))
    print('Targets with positive degree 0: ',len(pos_deg_0_targets))
    print('Targets with negative degree 0: ',len(neg_deg_0_targets))
    
    
    pos_annotated_ligands = list(set(ligands)-set(pos_deg_0_ligands))
    pos_annotated_targets = list(set(targets)-set(pos_deg_0_targets))
    
    neg_annotated_ligands = list(set(ligands)-set(neg_deg_0_ligands))
    neg_annotated_targets = list(set(targets)-set(neg_deg_0_targets))
    
    summat10 = np.loadtxt(open(summat10_file_path, "rb"), delimiter=",", skiprows=0) # Output of MATLAB run 
    summat01 = np.loadtxt(open(summat01_file_path, "rb"), delimiter=",", skiprows=0) # Output of MATLAB run 
    
    test_probabilty_predicted_conditioned = []

    ## Average conditional probability
    #conditoned_summat = np.divide(summat10,np.add(summat10,summat01)) # Elementwise pos_deg / (pos_deg + neg_deg)
    #conditoned_summat = conditoned_summat[~np.isnan(conditoned_summat)] 
    #average_conditional_probability = sum(conditoned_summat) / len(conditoned_summat) # Average over valid conditional probabilities
    p10_avg = np.mean(summat10)
    p01_avg = np.mean(summat01)
    average_conditional_probability = p10_avg / (p10_avg + p01_avg)
    
    
    drop_nan = []
    
    for index, row in tqdm(test.iterrows()):      
    
        if row['InChiKey'] in pos_annotated_ligands and row['target_aa_code'] in pos_annotated_targets:
            p10 = summat10[ligands.index(row['InChiKey']),targets.index(row['target_aa_code'])]
            p01 = summat01[ligands.index(row['InChiKey']),targets.index(row['target_aa_code'])]
            p10_conditioned = p10 / (p10 + p01)
        
        elif row['InChiKey'] in pos_annotated_ligands and row['target_aa_code'] not in pos_annotated_targets:
            p10_conditioned = ligand_degree_ratio[row['InChiKey']]['deg_ratio']  ## k_+ / (k_+ + k_-)
        
        elif row['InChiKey'] not in pos_annotated_ligands and row['target_aa_code'] in pos_annotated_targets:
            p10_conditioned = targets_degree_ratio[row['target_aa_code']]['deg_ratio'] ## k_+ / (k_+ + k_-)

        else:
            p10_conditioned = average_conditional_probability
            
        if math.isnan(p10_conditioned): 
            drop_nan.append(index)
        else:
            test_probabilty_predicted_conditioned.append(p10_conditioned)
            
        ## Performance on the test dataset

    print('AUC: ', roc_auc_score(test.drop(drop_nan)['Y'].tolist(), test_probabilty_predicted_conditioned))
    print('AUP: ', average_precision_score(test.drop(drop_nan)['Y'].tolist(), test_probabilty_predicted_conditioned))
        
    return roc_auc_score(test.drop(drop_nan)['Y'].tolist(), test_probabilty_predicted_conditioned), average_precision_score(test.drop(drop_nan)['Y'].tolist(), test_probabilty_predicted_conditioned)

### Generated the degree files - Unseen Nodes and Edges

In [19]:
for fold in tqdm(range(5)):

    train = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/train_' + str(fold) + '.csv')
    edges_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/test_unseen_edges_' + str(fold) + '.csv')
    nodes_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/test_unseen_nodes_' + str(fold) + '.csv')
    dump_degree_sequences(train,nodes_test,fold=fold,unseen_folder='VecNet_Unseen_Nodes')

100%|██████████| 2705/2705 [00:08<00:00, 316.16it/s]

100%|██████████| 3049/3049 [00:09<00:00, 320.25it/s]

100%|██████████| 2705/2705 [00:00<00:00, 1270218.58it/s]

100%|██████████| 2705/2705 [00:00<00:00, 1326349.35it/s]

100%|██████████| 3049/3049 [00:00<00:00, 1321665.24it/s]

100%|██████████| 2525/2525 [00:07<00:00, 326.04it/s]

100%|██████████| 2994/2994 [00:08<00:00, 333.85it/s]

100%|██████████| 2525/2525 [00:00<00:00, 1070516.28it/s]

100%|██████████| 2525/2525 [00:00<00:00, 1151654.81it/s]

100%|██████████| 2994/2994 [00:00<00:00, 1369437.97it/s]

100%|██████████| 3394/3394 [00:11<00:00, 283.67it/s]

100%|██████████| 3184/3184 [00:10<00:00, 299.84it/s]

100%|██████████| 3394/3394 [00:00<00:00, 1237006.24it/s]

100%|██████████| 3394/3394 [00:00<00:00, 1246756.68it/s]

100%|██████████| 3184/3184 [00:00<00:00, 1285585.67it/s]

100%|██████████| 2271/2271 [00:06<00:00, 344.63it/s]

100%|██████████| 2965/2965 [00:08<00:00, 348.61it/s]

100%|██████████| 2271/2271 [00:00<00:00, 11890

######## Now run the MATLAB code in Configuration Model - 5 fold folder to generate the matrices. #########

### Get Unseen Nodes and Edges Test Performance

In [4]:
auc_nodes = []
aup_nodes = []
auc_edges =[]
aup_edges = []


for fold in tqdm(range(5)):

    train = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/train_' + str(fold) + '.csv')
    edges_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/test_unseen_edges_' + str(fold) + '.csv')
    nodes_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/test_unseen_nodes_' + str(fold) + '.csv')

    ligand_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/Degree_Sequences/ligands_' + str(fold) + '.txt'
    target_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/Degree_Sequences/targets_' + str(fold) + '.txt'
    summat10_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/Degree_Sequences/summat10_' + str(fold) + '.csv'
    summat01_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Nodes/Degree_Sequences/summat01_' + str(fold) + '.csv'

    auc, aup = get_configuration_model_performance(train,nodes_test,ligand_file_path,target_file_path,summat10_file_path,summat01_file_path)
    auc_nodes.append(auc)
    aup_nodes.append(aup)
    
    auc, aup = get_configuration_model_performance(train,edges_test,ligand_file_path,target_file_path,summat10_file_path,summat01_file_path)
    auc_edges.append(auc)
    aup_edges.append(aup)

100%|██████████| 2705/2705 [00:00<00:00, 1015719.99it/s]

100%|██████████| 3049/3049 [00:00<00:00, 724516.06it/s]

100%|██████████| 2705/2705 [00:07<00:00, 361.63it/s]

100%|██████████| 3049/3049 [00:07<00:00, 388.77it/s]


Ligands with positive degree 0:  446
Ligands with negative degree 0:  69
Targets with positive degree 0:  2069
Targets with negative degree 0:  334



0it [00:00, ?it/s]
286it [00:00, 2851.66it/s]
574it [00:00, 2863.18it/s]
862it [00:00, 2868.63it/s]
1151it [00:00, 2874.87it/s]
1439it [00:00, 2859.62it/s]
1725it [00:00, 2850.90it/s]
2012it [00:00, 2854.59it/s]
2298it [00:00, 2852.78it/s]
2585it [00:00, 2855.48it/s]
2871it [00:01, 2854.16it/s]
3157it [00:01, 2851.85it/s]
3443it [00:01, 2853.74it/s]
3729it [00:01, 2831.36it/s]
4013it [00:01, 2827.45it/s]
4300it [00:01, 2838.95it/s]
4585it [00:01, 2841.07it/s]
4875it [00:01, 2857.58it/s]
5161it [00:01, 2850.12it/s]
5447it [00:01, 2841.88it/s]
5732it [00:02, 2818.98it/s]
6017it [00:02, 2827.18it/s]
6491it [00:02, 2841.40it/s]

100%|██████████| 2705/2705 [00:00<00:00, 1207491.73it/s]

100%|██████████| 3049/3049 [00:00<00:00, 907560.35it/s]

  1%|▏         | 39/2705 [00:00<00:06, 380.95it/s]

AUC:  0.5
AUP:  0.482976428901556



100%|██████████| 2705/2705 [00:07<00:00, 361.07it/s]

100%|██████████| 3049/3049 [00:07<00:00, 381.35it/s]


...

AUC:  0.5
AUP:  0.4646845759130697


100%|██████████| 3441/3441 [00:10<00:00, 318.93it/s]

100%|██████████| 3236/3236 [00:09<00:00, 337.96it/s]


Ligands with positive degree 0:  797
Ligands with negative degree 0:  83
Targets with positive degree 0:  2065
Targets with negative degree 0:  239



0it [00:00, ?it/s]
242it [00:00, 2413.55it/s]
505it [00:00, 2540.16it/s]
768it [00:00, 2579.77it/s]
1028it [00:00, 2585.22it/s]
1290it [00:00, 2595.14it/s]
1550it [00:00, 2590.80it/s]
1810it [00:00, 2535.61it/s]
2077it [00:00, 2576.21it/s]
2343it [00:00, 2601.28it/s]
2608it [00:01, 2615.86it/s]
2875it [00:01, 2630.40it/s]
3139it [00:01, 2569.18it/s]
3404it [00:01, 2592.03it/s]
3696it [00:01, 2585.17it/s]
100%|██████████| 5/5 [05:14<00:00, 62.91s/it]

AUC:  0.7602582697659274
AUP:  0.7580169573705


In [5]:
print('Unseen Nodes')
print('AUC: ', np.mean(auc_nodes), '+-', np.std(auc_nodes))
print('AUP: ', np.mean(aup_nodes), '+-', np.std(aup_nodes))

print('Unseen Edges')
print('AUC: ', np.mean(auc_edges), '+-', np.std(auc_edges))
print('AUP: ', np.mean(aup_edges), '+-', np.std(aup_edges))

Unseen Nodes
AUC:  0.5 +- 0.0
AUP:  0.4694108013484294 +- 0.014157461223848512
Unseen Edges
AUC:  0.7384689728991776 +- 0.014866946219643889
AUP:  0.738652990997668 +- 0.016840420839620104


### Generated the degree files - Unseen Targets

In [87]:
for fold in tqdm(range(5)):

    train = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/train_' + str(fold) + '.csv')
    edges_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/test_unseen_edges_' + str(fold) + '.csv')
    nodes_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/test_unseen_nodes_' + str(fold) + '.csv')
    dump_degree_sequences(train,nodes_test,fold=fold,unseen_folder='VecNet_Unseen_Targets')

100%|██████████| 6751/6751 [00:40<00:00, 164.89it/s]

100%|██████████| 3594/3594 [00:19<00:00, 183.33it/s]

100%|██████████| 6751/6751 [00:00<00:00, 1206619.78it/s]

100%|██████████| 6751/6751 [00:00<00:00, 1324713.28it/s]

100%|██████████| 3594/3594 [00:00<00:00, 1188920.94it/s]

100%|██████████| 6765/6765 [00:39<00:00, 170.49it/s]

100%|██████████| 3533/3533 [00:19<00:00, 185.24it/s]

100%|██████████| 6765/6765 [00:00<00:00, 1227110.09it/s]

100%|██████████| 6765/6765 [00:00<00:00, 1350650.54it/s]

100%|██████████| 3533/3533 [00:00<00:00, 1335479.09it/s]

100%|██████████| 6755/6755 [00:39<00:00, 170.48it/s]

100%|██████████| 3547/3547 [00:18<00:00, 187.09it/s]

100%|██████████| 6755/6755 [00:00<00:00, 868962.54it/s]

100%|██████████| 6755/6755 [00:00<00:00, 835743.00it/s]

100%|██████████| 3547/3547 [00:00<00:00, 870367.77it/s]

100%|██████████| 6753/6753 [00:39<00:00, 169.25it/s]

100%|██████████| 3527/3527 [00:18<00:00, 187.07it/s]

100%|██████████| 6753/6753 [00:00<00:00, 1167475.

### Get Unseen Targets Test Performance

In [12]:
auc_targets = []
aup_targets = []

for fold in tqdm(range(5)):

    train = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/train_' + str(fold) + '.csv')
    edges_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/test_unseen_edges_' + str(fold) + '.csv')
    nodes_test = pd.read_csv('../data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/test_unseen_nodes_' + str(fold) + '.csv')

    ligand_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/Degree_Sequences/ligands_' + str(fold) + '.txt'
    target_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/Degree_Sequences/targets_' + str(fold) + '.txt'
    summat10_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/Degree_Sequences/summat10_' + str(fold) + '.csv'
    summat01_file_path = '/data/sars-busters-consolidated/GitData/VecNet_Unseen_Targets/Degree_Sequences/summat01_' + str(fold) + '.csv'

    
    try:
        auc, aup = get_configuration_model_performance(train,nodes_test,ligand_file_path,target_file_path,summat10_file_path,summat01_file_path)
        auc_targets.append(auc)
        aup_targets.append(aup)
    except:
        continue

100%|██████████| 6751/6751 [00:00<00:00, 982060.36it/s]

100%|██████████| 3594/3594 [00:00<00:00, 515097.51it/s]

100%|██████████| 6751/6751 [00:40<00:00, 168.52it/s]

100%|██████████| 3594/3594 [00:19<00:00, 187.44it/s]


Ligands with positive degree 0:  1487
Ligands with negative degree 0:  219
Targets with positive degree 0:  621
Targets with negative degree 0:  0



0it [00:00, ?it/s]
188it [00:00, 1870.42it/s]
376it [00:00, 1853.86it/s]
568it [00:00, 1880.90it/s]
757it [00:00, 1876.89it/s]
945it [00:00, 1861.92it/s]
1132it [00:00, 1853.83it/s]
1318it [00:00, 1840.52it/s]
1503it [00:00, 1823.74it/s]
1692it [00:00, 1843.21it/s]
1880it [00:01, 1852.13it/s]
2072it [00:01, 1869.34it/s]
2259it [00:01, 1869.45it/s]
2452it [00:01, 1885.62it/s]
2641it [00:01, 1846.44it/s]
2826it [00:01, 1844.32it/s]
3021it [00:01, 1874.14it/s]
3209it [00:01, 1871.18it/s]
3406it [00:01, 1898.91it/s]
3598it [00:01, 1902.47it/s]
3789it [00:02, 1863.78it/s]
3984it [00:02, 1886.77it/s]
4173it [00:02, 1883.32it/s]
4366it [00:02, 1894.77it/s]
4556it [00:02, 1884.08it/s]
4747it [00:02, 1889.73it/s]
4939it [00:02, 1897.57it/s]
5129it [00:02, 1892.81it/s]
5319it [00:02, 1886.41it/s]
5508it [00:02, 1879.71it/s]
5696it [00:03, 1875.69it/s]
5887it [00:03, 1883.35it/s]
6078it [00:03, 1890.09it/s]
6269it [00:03, 1895.76it/s]
6491it [00:03, 1875.92it/s]
  0%|          | 0/6765 [00:00<?,

AUC:  0.7866021104337396
AUP:  0.739409327085565


100%|██████████| 6765/6765 [00:00<00:00, 1066108.08it/s]

100%|██████████| 3533/3533 [00:00<00:00, 867439.91it/s]

100%|██████████| 6765/6765 [00:39<00:00, 172.72it/s]

100%|██████████| 3533/3533 [00:18<00:00, 188.60it/s]


...

Ligands with positive degree 0:  1796
Ligands with negative degree 0:  200
Targets with positive degree 0:  613
Targets with negative degree 0:  0


100%|██████████| 6744/6744 [00:00<00:00, 1236779.60it/s]

100%|██████████| 3518/3518 [00:00<00:00, 863858.17it/s]

100%|██████████| 6744/6744 [00:38<00:00, 175.92it/s]

100%|██████████| 3518/3518 [00:18<00:00, 192.99it/s]


Ligands with positive degree 0:  1815
Ligands with negative degree 0:  210
Targets with positive degree 0:  590
Targets with negative degree 0:  0



0it [00:00, ?it/s]
195it [00:00, 1945.41it/s]
390it [00:00, 1915.60it/s]
583it [00:00, 1917.46it/s]
777it [00:00, 1925.55it/s]
982it [00:00, 1968.27it/s]
1179it [00:00, 1967.18it/s]
1376it [00:00, 1961.03it/s]
1579it [00:00, 1981.36it/s]
1778it [00:00, 1979.51it/s]
1976it [00:01, 1973.20it/s]
2174it [00:01, 1966.98it/s]
2374it [00:01, 1974.83it/s]
2574it [00:01, 1979.33it/s]
2772it [00:01, 1960.16it/s]
2969it [00:01, 1936.65it/s]
3163it [00:01, 1931.55it/s]
3358it [00:01, 1936.94it/s]
3556it [00:01, 1948.59it/s]
3751it [00:01, 1945.27it/s]
3949it [00:02, 1953.59it/s]
4148it [00:02, 1961.73it/s]
4345it [00:02, 1951.70it/s]
4541it [00:02, 1939.27it/s]
4735it [00:02, 1938.52it/s]
4929it [00:02, 1928.56it/s]
5128it [00:02, 1945.08it/s]
5323it [00:02, 1906.72it/s]
5520it [00:02, 1922.60it/s]
5721it [00:02, 1945.11it/s]
5916it [00:03, 1920.67it/s]
6111it [00:03, 1929.02it/s]
6308it [00:03, 1940.79it/s]
6626it [00:03, 1944.99it/s]
100%|██████████| 5/5 [07:31<00:00, 90.25s/it]

AUC:  0.7251730645670953
AUP:  0.6325534590013489


In [13]:
print('Unseen Targets')
print('AUC: ', np.mean(auc_targets), '+-', np.std(auc_targets))
print('AUP: ', np.mean(aup_targets), '+-', np.std(aup_targets))

Unseen Targets
AUC:  0.7548768473474594 +- 0.02187252726003232
AUP:  0.6916142410865558 +- 0.038474651918511626
